# Requirement

In [ ]:
import numpy as np
import pandas as pd
import tensorflow as tsflw
import h5py as h5
import xmljson as xml
import scipy as sp
#import scikit-learn as sktl
import tqdm as tq
import xarray as xa
import seaborn as sb
import openpyxl as opyx

print("numpy : ",np.__version__, "doit etre >= 1.14.3")
print("pandas : ",pd.__version__, "doit etre >= 0.22") #PPPPPPPPPPPPPRRRRRRRRRRRROOOOOOOOOOBBBBBBBBBBLLLLLLLLMMMMMMMMMM
print("tensorflow: ",tsflw.__version__, "doit etre = 2.2")
print("h5py: ",h5.__version__, "doit etre >= 2.8")
print("xmljson : ",xml.__version__, "doit etre >= 0.1.9")
print("scipy : ",sp.__version__, "doit etre >= 1.1")
#print("scikit-learn: ",sktl.__version__, "doit etre >= 0.20")
print("tqdm : ",tq.__version__, "doit etre >= 4.26")
print("xarray : ",xa.__version__, "doit etre >= 0.11.2")
print("seaborn : ",sb.__version__, "doit etre >= 0.9")
print("openpyxl : ",opyx.__version__, "doit etre >= 3.0")

# Test du git

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

### Etude des données du jeu d'entrainement

Ouverture du fichier 'ecg_tracings.hdf5' contenant un seul jeu de données nommé tracings. Cet ensemble de données est un tenseur (827, 4096, 12). La première dimension correspond aux 827 examens différents de différents patients ; la deuxième dimension correspond aux 4096 échantillons de signal ; la troisième dimension aux 12 dérivations différentes des examens ECG dans l'ordre suivant : {DI, DII, DIII, AVL, AVF, AVR, V1, V2, V3, V4, V5, V6}.

In [ ]:
import h5py
path = "data/"
with h5py.File(path+'ecg_tracings.hdf5', "r") as f:
    u = np.array(f['tracings'])
print(u.shape)

Ouverture du fichier 'attributes.csv' qui contient les attributs de base du patient : sexe (M ou F) et âge. Il contient 827 lignes (plus l'en-tête). Le i-ème tracé dans ecg_tracings.hdf5 correspond à la i-ème ligne.

In [ ]:
path = "data/"
attribute = pd.read_csv(path+'attributes.csv')
print(attribute.shape)
display(attribute.head())

Ouverture du fichier 'cardiologist1.csv' qui contient des annotations du premier cardiologue.

In [ ]:
path = "data/annotations/"
cardio1 = pd.read_csv(path+'cardiologist1.csv')
print(cardio1.shape)
display(cardio1.head())

Ouverture du fichier 'cardiologist2.csv' qui contient des annotations du deuxième cardiologue.

In [ ]:
path = "data/annotations/"
cardio2 = pd.read_csv(path+'cardiologist2.csv')
print(cardio2.shape)
display(cardio2.head())

Ouverture du fichier 'dnn.csv' qui contient les prédiction à partir du réseau de neurones profonds décrit dans "Diagnostic automatique de l'ECG de courte durée à 12 dérivations à l'aide d'un réseau de neurones profonds". Le seuil est fixé de manière à maximiser le score F1.

In [ ]:
path = "data/annotations/"
dnn = pd.read_csv(path+'dnn.csv')
print(dnn.shape)
display(dnn.head())

Ouverture du fichier 'cardiology_residents.csv' qui contient les annotations de deux résidents en cardiologie de 4e année (chacun annoté la moitié de l'ensemble de données).

In [ ]:
path = "data/annotations/"
cardio_resident = pd.read_csv(path+'cardiology_residents.csv')
print(cardio_resident.shape)
display(cardio_resident.head())

Ouverture du fichier 'emergency_residents.csv' qui contient les annotations de deux résidents d'urgence de 3e année (chacun annoté la moitié de l'ensemble de données).

In [ ]:
path = "data/annotations/"
emerg_resident = pd.read_csv(path+'emergency_residents.csv')
print(emerg_resident.shape)
display(emerg_resident.head())

Ouverture du fichier 'medical_students.csv' qui contient les annotations de deux étudiants en médecine de 5e année (chacun annoté la moitié de l'ensemble de données).

In [ ]:
path = "data/annotations/"
med_student = pd.read_csv(path+'medical_students.csv')
print(med_student.shape)
display(med_student.head())

Ouverture du fichier 'gold_standard.csv' contenant les annotation d'étalon-or pour cet ensemble de données de test. Lorsque le cardiologue 1 et le cardiologue 2 étaient d'accord, le diagnostic commun était considéré comme le gold standard. En cas de désaccord, un troisième spécialiste principal, au courant des annotations des deux autres, décidait du diagnostic.

In [ ]:
lead_order = ["DI", "DII", "DIII", "AVL", "AVF", "AVR", "V1", "V2", "V3", "V4", "V5", "V6"]
path = "data/annotations/"
df_goldStandard = pd.read_csv(path+'gold_standard.csv')
df_goldStandard['nb_abnormality'] =  df_goldStandard.sum(axis=1)
print(df_goldStandard.shape)
display(df_goldStandard.head(20).T)

In [ ]:
df_goldStandard.describe()

Visualisation des cardiogramme de 5 patient qui n'ont pas d'annomalies.

In [ ]:
for idx_abn, abnormality in enumerate(df_goldStandard.columns[:-1]):
    print(abnormality,idx_abn)
    
# visualisation a refaire en mieux
for idx_safe in df_goldStandard[df_goldStandard.nb_abnormality==0].index[:5]: #indice des gens qui ont un cardio safe
    fig, axs = plt.subplots(1, 3,figsize=(40,5))
    for i,pos in enumerate([1,6,11]):
        ax = axs[i]
        ax.set_title(lead_order[pos],fontsize=10)
        ax.plot(np.linspace(0,10.24,4096),u[idx_safe,:,pos],'k', linewidth=.5)
    plt.show()

Visualisation des cardiogrammes des patient qui ont une anomalie.

In [ ]:
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]
for abnormality in list_abnormality :
    print(abnormality, ' : ')
    for idx_anomal in df_goldStandard[(df_goldStandard[abnormality]==1)&(df_goldStandard.nb_abnormality==1)].index[:1] : #trouve l'indice ou le patient a l'anomalie
        fig, axs = plt.subplots(1, 3,figsize=(40,5))
        for i,pos in enumerate([1,6,11]):
            ax = axs[i]
            ax.set_title(lead_order[pos],fontsize=10)
            ax.plot(np.linspace(0,10.24,4096),u[idx_safe,:,pos],'k', linewidth=.5)
        plt.show()

Pourcentage de patient sans anomalie et atteind d'une certaine anomalie.

In [ ]:
#Patient sans anomalie
nb_patient = (df_goldStandard.nb_abnormality==0).sum()
print(nb_patient*100/827,'% de patient sans anomalie')

#Patient avec anomalie
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]
for abnormality in list_abnormality :
    nb_patient = df_goldStandard[abnormality].sum()
    print(nb_patient*100/827,'% de patient atteind de ', abnormality)

maladie en fonction de l'age

In [ ]:
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]
at = attribute["age"]
for index, abnormality in enumerate (list_abnormality) :
    mal = df_goldStandard[abnormality]
    plt.plot(at,mal,'.')
    plt.xlabel("age")
    plt.ylabel(abnormality)
    plt.show()

Nombre de malade en fonction age

In [ ]:
at_age = attribute["age"]
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]

for index, abnormality in enumerate (list_abnormality) :
    tab_age = np.zeros((99))
    for age in range (98):
        tab = []
        for i in range(827):
            if at_age[i] == age :
                tab.append(i)
        #print(tab) 
        if tab != []:
            compt = 0
            for j in range (len(tab)):
                if (df_goldStandard[abnormality][tab[j]] == 1):
                    compt = compt+1
            tab_age[age] = compt
    z = np.linspace(0,99,99)
    plt.plot(z,tab_age)
    plt.legend(list_abnormality)
plt.xlabel("age")
plt.ylabel("nombre de malade")
plt.show()

Nombre de malade en fonction sex

In [ ]:
at_sex = attribute["sex"]
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]

men_means = []
women_means = []
for index, abnormality in enumerate (list_abnormality) :
    tab_sex = np.zeros((2))
    for sex in ["M","F"]:
        tab = []
        for i in range(827):
            if at_sex[i] == sex :
                tab.append(i)
        #print(tab) 
        if tab != []:
            compt = 0
            for j in range (len(tab)):
                if (df_goldStandard[abnormality][tab[j]] == 1):
                    compt = compt+1
        if sex == "M":
            tab_sex[0] = compt
        if sex == "F":
            tab_sex[1] = compt
    men_means.append(tab_sex[0])
    women_means.append(tab_sex[1])
x = np.arange(len(list_abnormality))
width = 0.35
fig,ax = plt.subplots()
rects1 = ax.bar(x - width/2, men_means, width, label='Men')
rects2 = ax.bar(x + width/2, women_means, width, label='Women')
plt.xlabel("age")
plt.ylabel("nombre de malade")
ax.set_xticks(x)
ax.set_xticklabels(list_abnormality)
ax.legend()
plt.show()

### Utilisation du réseau de neurones pour prévoir les anomalies (pathologies cardiaques)

##### Voir pour ajouter un train.py

Exécution du programme 'predict.py' pour tester le modele modele.hdf5. Renvoie un fichier dnn_output.npy. 

In [ ]:
%run -i  'predict.py' 'data/ecg_tracings.hdf5' 'dnn_predicts_hdf5/model.hdf5' 

Affichage de dnn_predict

In [ ]:
df_ndarray = np.round(np.load("dnn_output.npy"),5)
dnn_out = pd.DataFrame(data = df_ndarray, columns = ["1dAVb","RBBB","LBBB","SB","AF","ST"])
dnn_out.head(11)

Exucution du programme 'generate_figures_and_tables.py' qui génère des figures et des tableaux à partir de l'article "Automatic Diagnosis o the Short-Duration12-Lead ECG using a Deep Neural Network". C'est-à-dire ......................................................................................................................................

Faire du blabla !!!!!

In [ ]:
%run -i 'generate_figures_and_tables.py'

Exécution du programme 'model.py' qui permet l'implémentation de l'architecture de réseau neuronal dans Keras. Le modèle utilisé dans l'article est un neurone résiduel. Le modèle reçoit un tenseur d'entrée de dimension (N, 4096, 12) et renvoie un tenseur de sortie de dimension (N, 6), pour lequel N est la taille du lot.

In [ ]:
%run -i 'model.py'

### Evaluation de la performance de notre classifieur

#### Pourcentage d'erreur  des prédictions faites par les différents médecins et le dnn d'antonio

In [ ]:
list_med = [cardio1 , cardio2 , cardio_resident , emerg_resident , med_student, dnn ]
list_med_str = ['cardio1', 'cardio2', 'cardio_resident', 'emerg_resident', 'med_student', 'dnn']
list_abnormality = ["1dAVb", "RBBB", "LBBB", "SB", "AF", "ST"]

for i,med in enumerate (list_med) :      
    compt = 0
    for patient in range (827):
        for abn in list_abnormality:
            if (df_goldStandard[abn][patient] != med[abn][patient]):
                compt = compt + 1
    print('Pourcentage d erreur du',list_med_str[i], ':', compt*100/827,'%' )
    

COMMENTAIRES ANGEL

#### Comparaison avec autre méthode de prédiction

In [ ]:
from sklearn.model_selection import train_test_split


In [ ]:
x_train,x_test = train_test_split(u)
y_train,y_test = train_test_split(df_goldStandard["nb_abnormality"])

print(x_train.shape)
nsamples, nx, ny = x_train.shape
x_train = x_train.reshape((nsamples,nx*ny))
nsamples, nx, ny = x_test.shape
x_test = x_test.reshape((nsamples,nx*ny))

print(x_train.shape,x_test.shape)
print(y_train.shape,y_test.shape)

##### Méthode du K-means (NON supervisée)

In [ ]:
from sklearn.cluster import KMeans
from sklearn.metrics import confusion_matrix

k = 2
kmeans = KMeans(n_clusters = k, random_state = 42)
kmeans.fit(x_train)

#help(confusion_matrix) 
Faut faire help de la matrice de confusion pour expliquer chaque etape. 

In [ ]:
pred = kmeans.fit_predict(x_test)
cm = confusion_matrix(y_test, pred)
print("Pourcentage d'erreur : ", ((cm[1][0]+cm[0][1])*100/207),'%')

##### Méthode de Random Forest (supervisée)

In [ ]:
from sklearn.ensemble import RandomForestClassifier

modelRF = RandomForestClassifier(class_weight='balanced')
modelRF.fit(x_train,y_train)
y_predRF = modelRF.predict(x_test)

In [ ]:
cm = confusion_matrix(y_test, y_predRF)
print("Pourcentage d'erreur : ", ((cm[1][0]+cm[0][1])*100/207),'%')